# SP3 Files and Orbit Interpolation

To process DORIS data we need to be able to compute (or estimate) the satellite position at the measurement epochs. The most simple case would be to interpolate satellite coordinates at every requested epoch, using the tabulated position (an/or velocity) vectors contained in a high-quality Sp3 file.